In [2]:
#!pip install openai

from openai import OpenAI
import json
import pandas as pd

client = OpenAI(api_key="YOUR_API_KEY")

# Forcing GPT to Return JSON Output

Sometimes, when you're using GPT in an application, it's helpful to get the response in a **structured format**, like **JSON**.  
This makes it easier for your program to read and use the result.

In this example, we tell GPT to return a JSON object by using the `response_format` parameter:


In [3]:
response = client.chat.completions.create(
    model="gpt-4o",
    response_format={"type": "json_object"}, # on force le format JSON
    messages=[
        {
            "role": "system",
            "content": (
                "Analyze the customer's message and return a JSON object with two fields: 'tag' and 'sentiment'."
            )
        },
        {
            "role": "user",
            "content": "Why was my package not delivered yet?"
        }
    ]
)

print(response.choices[0].message.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************bBMA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

#### Tip: Be Clear About the Expected Format

When you use:

`response_format={"type": "json_object"}`

you are telling the OpenAI API that you want the response to be in JSON format.
However, the model also needs to be told in the prompt itself what kind of answer you expect.

✅ So it's important to include a clear instruction in the system message, like:
`"Convert the user's request into a JSON object with two fields: 'intent' and 'topic'."
`

# Being Precise Helps Control the Output

To get reliable and structured answers from ChatGPT, it's important to **be as clear and specific as possible** in the system message.

*c'est important de mettre un exemple de ce qu'on attend comme output *

In [ ]:
response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    """
                    Analyze the customer's message and return a JSON object with 3 fields: 'tag', 'sentiment' and 'emergency'.

                    - 'tag' should describe the main topic of the message. It must be one of the following values: billing, technical_support, product_feedback, shipping, cancellation, other.
                    - 'sentiment' should describe the tone of the email and must be one of the following values: positive, neutral, negative.
                    - 'emergency' should describe the emergency of the message. It must be one of the following values: hign, medium, low

                    Example output:
                    {
                        "tag": "<the tag of the message>",
                        "sentiment": "<the tone of the message>",
                        "emergency": "<the emergency of the message>"
                    }
                    """
                )
            },
            {
                "role": "user",
                "content": "I am really fed up, can you please ship my parcel ASAP?"
            }
        ]
    )


print(response.choices[0].message.content)


{
    "tag": "shipping",
    "sentiment": "negative",
    "emergency": "high"
}


We improve control over the response by:


* Defining exactly which fields we want in the JSON (intent, topic)
* Listing the allowed values for intent
* Giving a clear example of what the output should look like






# 📬 Classifying Many Emails Using a Function

Now that we have a function called `def analyze_messages(input_message)` that takes one message and returns a **tag** and a **sentiment**, it's very easy to use it for **lots of messages**.

In [ ]:


def analyze_message(input_message):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": (
                    """
                    Analyze the customer's message and return a JSON object with 3 fields: 'tag', 'sentiment' and 'urgency'.

                    - 'tag' should describe the main topic of the message. It must be one of the following values: billing, technical_support, product_feedback, cancellation, other.
                    - 'sentiment' should describe the tone of the message and must be one of the following values: positive, neutral, negative.
                    - 'urgency' should describe the emergency of the message. It must be one of the following values: hign, medium, low



                    Example output:
                    {
                        "tag": "<the tag of the message>",
                        "sentiment": "<the tone of the message>",
                        "urgency": "<the emergency of the message>"
                    }
                    """
                )
            },
            {
                "role": "user",
                "content": input_message
            }
        ]
    )


    result = json.loads(response.choices[0].message.content)
    return result["tag"], result["sentiment"], result["urgency"]


We just need to:
1. Put all our messages in a list.
2. Call the function on each message using a simple loop.
3. Store the results in a table (here, we use a `DataFrame` from the `pandas` library).

This is useful when a company receives many customer emails and wants to **automatically detect the topic and tone** of each one — for example, to prioritize support tickets or understand customer feedback.


In [ ]:
messages = [
    "I just wanted to say how much I love your new app design—great job!",
    "Can you please confirm when my next payment is due?",
    "My internet keeps disconnecting every few minutes, and it's really annoying.",
    "I’ve enjoyed the service, but I no longer need it. Please cancel my subscription."
]


res = pd.DataFrame(columns=["message","tag", "sentiment", "urgency"])
for message in messages:
    tag, sentiment, urgency = analyze_message(message)
    new_row = pd.DataFrame({"message": [message],
                            "tag": [tag],
                            "sentiment": [sentiment],
                            "urgency":[urgency]})
    res = pd.concat([res, new_row], ignore_index=True)

res

,message,tag,sentiment,urgency
0,I just wanted to say how much I love your new ...,product_feedback,positive,low
1,Can you please confirm when my next payment is...,billing,neutral,low
2,My internet keeps disconnecting every few minu...,technical_support,negative,high
3,"I’ve enjoyed the service, but I no longer need...",cancellation,positive,medium


# ✏️ Practice Questions

1. **Update the list of tags**  
   Modify the system message so that the allowed tags are now:  
   `"billing"`, `"technical_support"`, `"product_feedback"`, `"shipping"`, `"cancellation"`, and `"other"`  
   ➕ This introduces a new category: `"shipping"`

2. **Add a third field: `urgency` (option)**  
   Extend the system message to request a new field in the JSON output: `"urgency"`  
   This field should describe how urgent the message is, with one of the following values:  
   `"high"`, `"medium"`, or `"low"`

   ✏️ For example, a message like  
   *"I’ve been overcharged and no one is replying to my emails!"*  
   would likely be:  
   `{"tag": "billing", "sentiment": "negative", "urgency": "high"}`